<a href="https://colab.research.google.com/github/pablohenrique93/projeto-treino-etl-pyspark-covid-vaccination/blob/main/pyspark_etl_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de Bibliotecas

In [ ]:
!pip install pyspark

In [54]:
pip -q install plotly --upgrade

In [ ]:
pip install pymongo

# Importação de Bibliotecas

In [55]:
# Importando pyspark
import pyspark
# Importando a SparkSession
from pyspark.sql import SparkSession
# Importando pyspark sql funções
import pyspark.sql.functions as F
# Importando os tipos
from pyspark.sql.types import *
# Importando o Pandas
import pandas as pd

In [56]:
# Importando funções
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql.functions import to_date
from pyspark.sql.functions import regexp_replace, split
from pyspark.sql.functions import *
from pyspark.sql.functions import min, max, col, date_format

In [57]:
# Importando Bibliotecas para Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [58]:
# Importação da Biblioteca para acesso ao MongoDB
import pymongo
from pymongo import MongoClient

# Configuração da Spark Session

In [59]:
# Configurando a variável do ambiente da sessão spark
spark = (
    SparkSession.builder
                .master('local')
                .appName('etl_spark')
                .config('spark.ui.port','4050')
                .getOrCreate()
)

In [ ]:
# Verificando a Spark Session
spark

# Carregamento da Base de Dados

In [60]:
# Criando o Dataframe 
df = (
    spark.createDataFrame(pd.read_csv('/content/vaccination-data.csv' ))
         
)

In [ ]:
# Visualização do Dataframe
df.show()

# Backup do Dataframe


In [ ]:
df_backup = df

# Pré Análise do Dataframe

In [ ]:
# Verificando as colunas bem como seus tipos
df.printSchema()

In [ ]:
# Verificando o tamanho do dataframe ou quantidade de linhas
tamanho = df.count()
print("O tamanho do DataFrame é:", tamanho)

In [ ]:
# Verificando o resumo estatístico do dataframe, ou seja, um resumo geral dos registros presentes em cada coluna
df.summary().show()

In [ ]:
# Verificando a quantidade de linhas duplicadas
total_linhas = df.count()
total_linhas_distintas = df.distinct().count()
print(total_linhas - total_linhas_distintas)
# Com podemos observar o dataframe não possui linhas duplicadas

In [ ]:
# Verificando a quantidade de valores nulos em cada coluna
quantidade_nulos = df.select([spark_sum(col(coluna).isNull().cast("int")).alias(coluna) for coluna in df.columns])
# Exibindo a quantidade de valores nulos em cada coluna
quantidade_nulos.show()

In [ ]:
# Exibindo os tipos de cada coluna
df.dtypes

# Tratamento da Base de Dados

In [ ]:
df.show()

In [ ]:
# Convertendo as colunas 'DATE_UPDATED' e 'FIRST_VACCINE_DATE' para o tipo date
df1 = df.withColumn("data_ultima_atualizacao", to_date("DATE_UPDATED", "yyyy-MM-dd"))
df2 = df1.withColumn("data_inicio_vacinacao", to_date("FIRST_VACCINE_DATE", "yyyy-MM-dd"))

In [ ]:
# Drop das colunas 'DATE_UPDATED' e 'FIRST_VACCINE_DATE' pois ja foram convertidas para o formato date com a criação das colunas anteriormente
df3 = df2.drop("DATE_UPDATED")
df4 = df3.drop("FIRST_VACCINE_DATE")

In [ ]:
# Renomeação e tradução das colunas com o objetivo de facilitar a compreensão dos dados do dataframe
df5 = df4.toDF(*['pais', 'codigo_pais', 'escritorio_oms', 'fonte_dos_dados', 'total_cumulativo_doses_vacina', 
                 'cumulativo_de_pessoas_com_uma_dose', 'cumulativo_de_doses_por_100hab', 
                 'cumulativo_de_pessoas_com_uma_dose_por_100hab', 'cumulativo_de_pessoas_totalmente_vacinadas',
                 'cumulativo_de_pessoas_totalmente_vacinadas_por_100hab', 'vacinas_usadas', 'numero_de_tipos_de_vacina_usados',
                 'pessoas_com_dose_adicional', 'pessoas_com_dose_adicional_por_100hab', 'data_ultima_atualizacao', 'data_inicio_vacinacao'])
df5.show()

In [ ]:
# Obtendo todos os dados da coluna "pais" como uma lista com o objetivo de exibir todos os países presentes e traduzi-los
dados_coluna = df5.select("pais").collect()

In [ ]:
# Exibindo todos os dados da coluna
for row in dados_coluna:
    print(row[0])

In [ ]:
# Renomeação e tradução dos registros da coluna país
df6 = df5.withColumn("pais", regexp_replace("pais","Afghanistan","Afeganistão")) \
                 .withColumn("pais", regexp_replace("pais","Albania","Albânia")) \
                 .withColumn("pais", regexp_replace("pais","Algeria","Argélia")) \
                 .withColumn("pais", regexp_replace("pais","American Samoa","Samoa Americana")) \
                 .withColumn("pais", regexp_replace("pais","Andorra","Andorra")) \
                 .withColumn("pais", regexp_replace("pais","Angola","Angola")) \
                 .withColumn("pais", regexp_replace("pais","Anguilla","Anguilla")) \
                 .withColumn("pais", regexp_replace("pais","Antigua and Barbuda","Antígua e Barbuda")) \
                 .withColumn("pais", regexp_replace("pais","Croatia","Croácia")) \
                 .withColumn("pais", regexp_replace("pais","Argentina","Argentina")) \
                 .withColumn("pais", regexp_replace("pais","Armenia","Armênia")) \
                 .withColumn("pais", regexp_replace("pais","Aruba","Aruba")) \
                 .withColumn("pais", regexp_replace("pais","Australia","Austrália")) \
                 .withColumn("pais", regexp_replace("pais","Austria","Áustria")) \
                 .withColumn("pais", regexp_replace("pais","Azerbaijan","Azerbaijão")) \
                 .withColumn("pais", regexp_replace("pais","Bahamas","Bahamas")) \
                 .withColumn("pais", regexp_replace("pais","Bahrain","Bahrein")) \
                 .withColumn("pais", regexp_replace("pais","Bangladesh","Bangladesh")) \
                 .withColumn("pais", regexp_replace("pais","Barbados","Barbados")) \
                 .withColumn("pais", regexp_replace("pais","Belarus","Bielorrússia")) \
                 .withColumn("pais", regexp_replace("pais","Belgium","Bélgica")) \
                 .withColumn("pais", regexp_replace("pais","Belize","Belize")) \
                 .withColumn("pais", regexp_replace("pais","Benin","Benin")) \
                 .withColumn("pais", regexp_replace("pais","Bermuda","Bermuda")) \
                 .withColumn("pais", regexp_replace("pais","Bhutan","Butão")) \
                 .withColumn("pais", regexp_replace("pais","Bolivia (Plurinational State of)","Bolívia")) \
                 .withColumn("pais", regexp_replace("pais","Bonaire","Bonaire")) \
                 .withColumn("pais", regexp_replace("pais","Bonaire, Sint Eustatius and Saba","Bonaire, Sint Eustatius and Saba")) \
                 .withColumn("pais", regexp_replace("pais","Bosnia and Herzegovina","Bósnia e Herzegovina")) \
                 .withColumn("pais", regexp_replace("pais","Botswana","Botsuana")) \
                 .withColumn("pais", regexp_replace("pais","Brazil","Brasil")) \
                 .withColumn("pais", regexp_replace("pais","British Virgin Islands","Ilhas Virgens Britânicas")) \
                 .withColumn("pais", regexp_replace("pais","Brunei Darussalam","Brunei Darussalam")) \
                 .withColumn("pais", regexp_replace("pais","Bulgaria","Bulgária")) \
                 .withColumn("pais", regexp_replace("pais","Burkina Faso","Burkina Faso")) \
                 .withColumn("pais", regexp_replace("pais","Burundi","Burundi")) \
                 .withColumn("pais", regexp_replace("pais","Cabo Verde","Cabo Verde")) \
                 .withColumn("pais", regexp_replace("pais","Cambodia","Cambodja")) \
                 .withColumn("pais", regexp_replace("pais","Cameroon","Camarões")) \
                 .withColumn("pais", regexp_replace("pais","Argentina","Argentina")) \
                 .withColumn("pais", regexp_replace("pais","Canada","Canadá")) \
                 .withColumn("pais", regexp_replace("pais","Cayman Islands","Ilhas Cayman")) \
                 .withColumn("pais", regexp_replace("pais","Central African Republic","República Centro-Africana")) \
                 .withColumn("pais", regexp_replace("pais","Austria","Áustria")) \
                 .withColumn("pais", regexp_replace("pais","Chad","Chade")) \
                 .withColumn("pais", regexp_replace("pais","Chile","Chile")) \
                 .withColumn("pais", regexp_replace("pais","China","China")) \
                 .withColumn("pais", regexp_replace("pais","Colombia","Colômbia")) \
                 .withColumn("pais", regexp_replace("pais","Comoros","Comores")) \
                 .withColumn("pais", regexp_replace("pais","Congo","Congo")) \
                 .withColumn("pais", regexp_replace("pais","Cook Islands","Ilhas Cook")) \
                 .withColumn("pais", regexp_replace("pais","Costa Rica","Costa Rica")) \
                 .withColumn("pais", regexp_replace("pais","Côte d’Ivoire","Costa do Marfim")) \
                 .withColumn("pais", regexp_replace("pais","Bermuda","Bermuda")) \
                 .withColumn("pais", regexp_replace("pais","Croatia","Croácia")) \
                 .withColumn("pais", regexp_replace("pais","Cuba","Cuba")) \
                 .withColumn("pais", regexp_replace("pais","Curaçao","Curaçao")) \
                 .withColumn("pais", regexp_replace("pais","Cyprus","Chipre")) \
                 .withColumn("pais", regexp_replace("pais","Czechia","República Tcheca")) \
                 .withColumn("pais", regexp_replace("pais","Democratic Republic of the Congo","República Democrática do Congo")) \
                 .withColumn("pais", regexp_replace("pais","Denmark","Dinamarca")) \
                 .withColumn("pais", regexp_replace("pais","Djibouti","Djibouti")) \
                 .withColumn("pais", regexp_replace("pais","Dominica","Dominica")) \
                 .withColumn("pais", regexp_replace("pais","Dominican Republic","República Dominicana")) \
                 .withColumn("pais", regexp_replace("pais","Ecuador","Equador")) \
                 .withColumn("pais", regexp_replace("pais","Egypt","Egito")) \
                 .withColumn("pais", regexp_replace("pais","El Salvador","El Salvador")) \
                 .withColumn("pais", regexp_replace("pais","Equatorial Guinea","Guiné Equatorial")) \
                 .withColumn("pais", regexp_replace("pais","Eritrea","Eritreia")) \
                 .withColumn("pais", regexp_replace("pais","Estonia","Estônia")) \
                 .withColumn("pais", regexp_replace("pais","Eswatini","Essuatíni")) \
                 .withColumn("pais", regexp_replace("pais","Ethiopia","Etiópia")) \
                 .withColumn("pais", regexp_replace("pais","Falkland Islands (Malvinas)","Ilhas Falkland (Malvinas)")) \
                 .withColumn("pais", regexp_replace("pais","Faroe Islands","Ilhas Faroe")) \
                 .withColumn("pais", regexp_replace("pais","Fiji","Fiji")) \
                 .withColumn("pais", regexp_replace("pais","Finland","Finlândia")) \
                 .withColumn("pais", regexp_replace("pais","France","França")) \
                 .withColumn("pais", regexp_replace("pais","French Guiana","Guiana Francesa")) \
                 .withColumn("pais", regexp_replace("pais","French Polynesia","Polinésia Francesa")) \
                 .withColumn("pais", regexp_replace("pais","Gabon","Gabão")) \
                 .withColumn("pais", regexp_replace("pais","Gambia","Gâmbia")) \
                 .withColumn("pais", regexp_replace("pais","Georgia","Geórgia")) \
                 .withColumn("pais", regexp_replace("pais","Germany","Alemanha")) \
                 .withColumn("pais", regexp_replace("pais","Ghana","Gana")) \
                 .withColumn("pais", regexp_replace("pais","Gibraltar","Gibraltar")) \
                 .withColumn("pais", regexp_replace("pais","Greece","Grécia")) \
                 .withColumn("pais", regexp_replace("pais","Greenland","Groenlândia")) \
                 .withColumn("pais", regexp_replace("pais","Grenada","Granada")) \
                 .withColumn("pais", regexp_replace("pais","Guadeloupe","Guadalupe")) \
                 .withColumn("pais", regexp_replace("pais","Guam","Guam")) \
                 .withColumn("pais", regexp_replace("pais","Guatemala","Guatemala")) \
                 .withColumn("pais", regexp_replace("pais","Guernsey","Guernsey")) \
                 .withColumn("pais", regexp_replace("pais","Guinea","Guiné")) \
                 .withColumn("pais", regexp_replace("pais","Guinea-Bissau","Guiné-Bissau")) \
                 .withColumn("pais", regexp_replace("pais","Guyana","Guiana")) \
                 .withColumn("pais", regexp_replace("pais","Haiti","Haiti")) \
                 .withColumn("pais", regexp_replace("pais","Honduras","Honduras")) \
                 .withColumn("pais", regexp_replace("pais","Hungary","Hungria")) \
                 .withColumn("pais", regexp_replace("pais","Iceland","Islândia")) \
                 .withColumn("pais", regexp_replace("pais","India","Índia")) \
                 .withColumn("pais", regexp_replace("pais","Indonesia","Indonésia")) \
                 .withColumn("pais", regexp_replace("pais","Iran (Islamic Republic of)","Irã")) \
                 .withColumn("pais", regexp_replace("pais","Iraq","Iraque")) \
                 .withColumn("pais", regexp_replace("pais","Ireland","Irlanda")) \
                 .withColumn("pais", regexp_replace("pais","Isle of Man","Ilha de Man")) \
                 .withColumn("pais", regexp_replace("pais","Israel","Israel")) \
                 .withColumn("pais", regexp_replace("pais","Italy","Itália")) \
                 .withColumn("pais", regexp_replace("pais","Jamaica","Jamaica")) \
                 .withColumn("pais", regexp_replace("pais","Japan","Japão")) \
                 .withColumn("pais", regexp_replace("pais","Jersey","Jersey")) \
                 .withColumn("pais", regexp_replace("pais","Jordan","Jordânia")) \
                 .withColumn("pais", regexp_replace("pais","Kazakhstan","Cazaquistão")) \
                 .withColumn("pais", regexp_replace("pais","Kenya","Quênia")) \
                 .withColumn("pais", regexp_replace("pais","Kiribati","Kiribati")) \
                 .withColumn("pais", regexp_replace("pais","Kosovo","Kosovo")) \
                 .withColumn("pais", regexp_replace("pais","Kuwait","Kuwait")) \
                 .withColumn("pais", regexp_replace("pais","Kyrgyzstan","Quirguistão"))

In [ ]:
df7 = df6.withColumn("pais", regexp_replace("pais","Lao People's Democratic Republic","República Democrática Popular do Laos")) \
                 .withColumn("pais", regexp_replace("pais","Latvia","Letônia")) \
                 .withColumn("pais", regexp_replace("pais","Lebanon","Líbano")) \
                 .withColumn("pais", regexp_replace("pais","Lesotho","Lesoto")) \
                 .withColumn("pais", regexp_replace("pais","Liberia","Libéria")) \
                 .withColumn("pais", regexp_replace("pais","Libya","Líbia")) \
                 .withColumn("pais", regexp_replace("pais","Liechtenstein","Liechtenstein")) \
                 .withColumn("pais", regexp_replace("pais","Lithuania","Lituânia")) \
                 .withColumn("pais", regexp_replace("pais","Luxembourg","Luxemburgo")) \
                 .withColumn("pais", regexp_replace("pais","Madagascar","Madagascar")) \
                 .withColumn("pais", regexp_replace("pais","Malawi","Malawi")) \
                 .withColumn("pais", regexp_replace("pais","Malaysia","Malásia")) \
                 .withColumn("pais", regexp_replace("pais","Maldives","Maldivas")) \
                 .withColumn("pais", regexp_replace("pais","Mali","Mali")) \
                 .withColumn("pais", regexp_replace("pais","Malta","Malta")) \
                 .withColumn("pais", regexp_replace("pais","Marshall Islands","Ilhas Marshall")) \
                 .withColumn("pais", regexp_replace("pais","Martinique","Martinica")) \
                 .withColumn("pais", regexp_replace("pais","Mauritania","Mauritânia")) \
                 .withColumn("pais", regexp_replace("pais","Mauritius","Maurício")) \
                 .withColumn("pais", regexp_replace("pais","Mexico","México")) \
                 .withColumn("pais", regexp_replace("pais","Micronesia (Federated States of)","Micronésia")) \
                 .withColumn("pais", regexp_replace("pais","Monaco","Mônaco")) \
                 .withColumn("pais", regexp_replace("pais","Mongolia","Mongólia")) \
                 .withColumn("pais", regexp_replace("pais","Montenegro","Montenegro")) \
                 .withColumn("pais", regexp_replace("pais","Montserrat","Montserrat")) \
                 .withColumn("pais", regexp_replace("pais","Morocco","Marrocos")) \
                 .withColumn("pais", regexp_replace("pais","Mozambique","Moçambique")) \
                 .withColumn("pais", regexp_replace("pais","Myanmar","Mianmar")) \
                 .withColumn("pais", regexp_replace("pais","Namibia","Namíbia")) \
                 .withColumn("pais", regexp_replace("pais","Nauru","Nauru")) \
                 .withColumn("pais", regexp_replace("pais","Nepal","Nepal")) \
                 .withColumn("pais", regexp_replace("pais","Netherlands","Holanda")) \
                 .withColumn("pais", regexp_replace("pais","New Caledonia","Nova Caledônia")) \
                 .withColumn("pais", regexp_replace("pais","New Zealand","Nova Zelândia")) \
                 .withColumn("pais", regexp_replace("pais","Nicaragua","Nicarágua")) \
                 .withColumn("pais", regexp_replace("pais","Niger","Níger")) \
                 .withColumn("pais", regexp_replace("pais","Nigeria","Nigéria")) \
                 .withColumn("pais", regexp_replace("pais","Niue","Niue")) \
                 .withColumn("pais", regexp_replace("pais","North Macedonia","Macedônia do Norte")) \
                 .withColumn("pais", regexp_replace("pais","Northern Mariana Islands (Commonwealth of the)","Ilhas Marianas do Norte")) \
                 .withColumn("pais", regexp_replace("pais","Norway","Noruega")) \
                 .withColumn("pais", regexp_replace("pais","occupied Palestinian territory","Território palestino ocupado")) \
                 .withColumn("pais", regexp_replace("pais","Oman","Omã")) \
                 .withColumn("pais", regexp_replace("pais","Pakistan","Paquistão")) \
                 .withColumn("pais", regexp_replace("pais","Palau","Palau")) \
                 .withColumn("pais", regexp_replace("pais","Panama","Panamá")) \
                 .withColumn("pais", regexp_replace("pais","Papua New Guinea","Papua Nova Guiné")) \
                 .withColumn("pais", regexp_replace("pais","Paraguay","Paraguai")) \
                 .withColumn("pais", regexp_replace("pais","Peru","Peru")) \
                 .withColumn("pais", regexp_replace("pais","Philippines","Filipinas")) \
                 .withColumn("pais", regexp_replace("pais","Pitcairn Islands","Ilhas Pitcairn")) \
                 .withColumn("pais", regexp_replace("pais","Poland","Polônia")) \
                 .withColumn("pais", regexp_replace("pais","Portugal","Portugal")) \
                 .withColumn("pais", regexp_replace("pais","Puerto Rico","Porto Rico")) \
                 .withColumn("pais", regexp_replace("pais","Qatar","Catar")) \
                 .withColumn("pais", regexp_replace("pais","Republic of Korea","República da Coréia")) \
                 .withColumn("pais", regexp_replace("pais","Republic of Moldova","República da Moldávia")) \
                 .withColumn("pais", regexp_replace("pais","Romania","Romênia")) \
                 .withColumn("pais", regexp_replace("pais","Russian Federation","Federação Russa")) \
                 .withColumn("pais", regexp_replace("pais","Rwanda","Ruanda")) \
                 .withColumn("pais", regexp_replace("pais","Saba","Sabá")) \
                 .withColumn("pais", regexp_replace("pais","Saint Helena, Ascension and Tristan da Cunha","Santa Helena, Ascensão e Tristão da Cunha")) \
                 .withColumn("pais", regexp_replace("pais","Saint Kitts and Nevis","São Cristóvão e Névis")) \
                 .withColumn("pais", regexp_replace("pais","Saint Lucia","Santa Lúcia")) \
                 .withColumn("pais", regexp_replace("pais","Saint Vincent and the Grenadines","São Vicente e Granadinas")) \
                 .withColumn("pais", regexp_replace("pais","Samoa","Samoa")) \
                 .withColumn("pais", regexp_replace("pais","San Marino","San Marino")) \
                 .withColumn("pais", regexp_replace("pais","Sao Tome and Principe","São Tomé e Príncipe")) \
                 .withColumn("pais", regexp_replace("pais","Saudi Arabia","Arábia Saudita")) \
                 .withColumn("pais", regexp_replace("pais","Senegal","Senegal")) \
                 .withColumn("pais", regexp_replace("pais","Serbia","Sérvia")) \
                 .withColumn("pais", regexp_replace("pais","Seychelles","Seychelles")) \
                 .withColumn("pais", regexp_replace("pais","Sierra Leone","Serra Leoa")) \
                 .withColumn("pais", regexp_replace("pais","Singapore","Singapura")) \
                 .withColumn("pais", regexp_replace("pais","Sint Eustatius","Santo Eustáquio")) \
                 .withColumn("pais", regexp_replace("pais","Sint Maarten","São Martinho")) \
                 .withColumn("pais", regexp_replace("pais","Slovakia","Eslováquia")) \
                 .withColumn("pais", regexp_replace("pais","Slovenia","Eslovênia")) \
                 .withColumn("pais", regexp_replace("pais","Solomon Islands","Ilhas Salomão")) \
                 .withColumn("pais", regexp_replace("pais","Somalia","Somália")) \
                 .withColumn("pais", regexp_replace("pais","South Africa","África do Sul")) \
                 .withColumn("pais", regexp_replace("pais","South Sudan","Sudão do Sul")) \
                 .withColumn("pais", regexp_replace("pais","Spain","Espanha")) \
                 .withColumn("pais", regexp_replace("pais","Sri Lanka","Sri Lanka")) \
                 .withColumn("pais", regexp_replace("pais","Sudan","Sudão")) \
                 .withColumn("pais", regexp_replace("pais","Suriname","Suriname")) \
                 .withColumn("pais", regexp_replace("pais","Sweden","Suécia")) \
                 .withColumn("pais", regexp_replace("pais","Switzerland","Suíça")) \
                 .withColumn("pais", regexp_replace("pais","Syrian Arab Republic","República Árabe da Síria")) \
                 .withColumn("pais", regexp_replace("pais","Tajikistan","Tajiquistão")) \
                 .withColumn("pais", regexp_replace("pais","Thailand","Tailândia")) \
                 .withColumn("pais", regexp_replace("pais","The United Kingdom","Reino Unido")) \
                 .withColumn("pais", regexp_replace("pais","Timor-Leste","Timor-Leste")) \
                 .withColumn("pais", regexp_replace("pais","Togo","Togo")) \
                 .withColumn("pais", regexp_replace("pais","Tokelau","Tokelau")) \
                 .withColumn("pais", regexp_replace("pais","Tonga","Tonga")) \
                 .withColumn("pais", regexp_replace("pais","Trinidad and Tobago","Trinidad e Tobago")) \
                 .withColumn("pais", regexp_replace("pais","Tunisia","Tunísia")) \
                 .withColumn("pais", regexp_replace("pais","Turkey","Turquia")) \
                 .withColumn("pais", regexp_replace("pais","Turkmenistan","Turcomenistão")) \
                 .withColumn("pais", regexp_replace("pais","Turks and Caicos Islands","Ilhas Turks e Caicos")) \
                 .withColumn("pais", regexp_replace("pais","Tuvalu","Tuvalu")) \
                 .withColumn("pais", regexp_replace("pais","Uganda","Uganda")) \
                 .withColumn("pais", regexp_replace("pais","Ukraine","Ucrânia")) \
                 .withColumn("pais", regexp_replace("pais","United Arab Emirates","Emirados Árabes Unidos")) \
                 .withColumn("pais", regexp_replace("pais","United Republic of Tanzania","República Unida da Tanzânia")) \
                 .withColumn("pais", regexp_replace("pais","United States of America","Estados Unidos da América")) \
                 .withColumn("pais", regexp_replace("pais","Uruguay","Uruguai")) \
                 .withColumn("pais", regexp_replace("pais","Uzbekistan","Uzbequistão")) \
                 .withColumn("pais", regexp_replace("pais","Vanuatu","Vanuatu")) \
                 .withColumn("pais", regexp_replace("pais","Venezuela (Bolivarian Republic of)","Venezuela")) \
                 .withColumn("pais", regexp_replace("pais","Viet Nam","Vietnã")) \
                 .withColumn("pais", regexp_replace("pais","Wallis and Futuna","Wallis e Futuna")) \
                 .withColumn("pais", regexp_replace("pais","Yemen","Iêmen")) \
                 .withColumn("pais", regexp_replace("pais","Zambia","Zâmbia")) \
                 .withColumn("pais", regexp_replace("pais","Zimbabwe","Zimbábue"))

In [ ]:
# Drop da coluna "codigo_pais" pois se torna irrelevante já que a própria coluna "país" ja possui 
# registros identificando de qual país se tratam os dados
df8 = df7.drop("codigo_pais")

In [ ]:
# Ordenação de colunas do Dataframe com o objetivo de facilitar o entendimento dos dados
df9 = df8.select(col("pais"), col("escritorio_oms"), col("fonte_dos_dados"), col("data_ultima_atualizacao"), col("data_inicio_vacinacao"), col("total_cumulativo_doses_vacina"), 
                 col("cumulativo_de_doses_por_100hab"), col("cumulativo_de_pessoas_com_uma_dose"), col("cumulativo_de_pessoas_com_uma_dose_por_100hab"), col("cumulativo_de_pessoas_totalmente_vacinadas"), col("cumulativo_de_pessoas_totalmente_vacinadas_por_100hab"), 
                 col("vacinas_usadas"), col("numero_de_tipos_de_vacina_usados"), col("pessoas_com_dose_adicional"), col("pessoas_com_dose_adicional_por_100hab"))

# Consultas

In [ ]:
# Visualizando a quantidade de registros de cada pais
df9.groupBy('pais').count().orderBy(F.col('count').desc()).show(100)

In [ ]:
# Verificando a quantidade de pessoas totalmente vacinadas por país

# Agrupamento por 'pais' e soma dos registros da coluna 'cumulativo_de_pessoas_totalmente_vacinadas'
df_grouped = df9.groupBy('pais').sum('cumulativo_de_pessoas_totalmente_vacinadas')

# Ordenação de forma decrescente e exibição do DataFrame resultante
df_grouped.orderBy(desc('sum(cumulativo_de_pessoas_totalmente_vacinadas)')).show(truncate=False)

In [ ]:
# Verificando a quantidade de tipos de vacinas utilizadas por país

df_grouped2 = df9.groupBy('pais').sum('numero_de_tipos_de_vacina_usados')
df_grouped2 = df_grouped2.na.drop()
df_grouped2.orderBy(desc('sum(numero_de_tipos_de_vacina_usados)')).show(truncate=False)

In [ ]:
# Verificando os países que iniciaram o período de vacinação primeiro

df9.groupBy('pais') \
   .agg(min('data_inicio_vacinacao').alias('min_data_inicio_vacinacao')) \
   .where(col('min_data_inicio_vacinacao').isNotNull()) \
   .orderBy('min_data_inicio_vacinacao') \
   .show(truncate=False)

In [ ]:
# Verificando o número total de pessoas com somente uma dose por país, ordenado pela ultima data de atualização

df9.groupBy('pais') \
   .agg(sum('cumulativo_de_pessoas_com_uma_dose').alias('total_pessoas_com_uma_dose'),
        max('data_ultima_atualizacao').alias('ultima_atualizacao')) \
   .orderBy(desc('ultima_atualizacao')) \
   .show()

In [ ]:
# Verificando o número total de pessoas com somente uma dose por país, ordenado pelo número total de pessoas

df9.groupBy('pais') \
   .agg(sum('cumulativo_de_pessoas_com_uma_dose').alias('total_pessoas_com_uma_dose'),
        max('data_ultima_atualizacao').alias('ultima_atualizacao')) \
   .orderBy(desc('total_pessoas_com_uma_dose')) \
   .show()

# Conversão de Pyspark para Pandas

In [ ]:
# Convertendo dataframe tratado para Pandas para etapa de visualização de dados
df_pandas = df9.toPandas()
# Exibindo os primeiros registros do dataset no Pandas
df_pandas.head()

In [ ]:
# Exibindo as duas ultimas linhas do dataset no Pandas
df_pandas.tail(2)

# Visualização de Dados

In [ ]:
# Conferindo os tipos das colunas
df_pandas.dtypes
# é possível observar que as colunas de data passaram para "object"

In [ ]:
# Realizando a conversão das mesmas para o tipo "date" para trabalhar nas plotagens da melhor forma
df_pandas['data_ultima_atualizacao'] = pd.to_datetime(df_pandas['data_ultima_atualizacao'])
df_pandas['data_inicio_vacinacao'] = pd.to_datetime(df_pandas['data_inicio_vacinacao'])

In [ ]:
# Plot com Seaborn para visualização dos registros da coluna 'fonte_dos_dados'
sns.countplot(x=df_pandas['fonte_dos_dados']);
# É possível observar que a maioria dos dados são informados pelos estados membros e/ou provenientes de relatórios oficiais.
# E uma pequena taxa são provenientes do "Our World in Data" 

In [ ]:
# Plotagem da quantidade de pessoas totalmente vacinadas por país

# Ordenando o DataFrame pela coluna "cumulativo_de_pessoas_totalmente_vacinadas"
df_sorted = df_pandas.sort_values(by='cumulativo_de_pessoas_totalmente_vacinadas', ascending=False)

# Plotando o gráfico de barras ordenado usando seaborn
sns.barplot(x='pais', y='cumulativo_de_pessoas_totalmente_vacinadas', data=df_sorted)

# Rotacionando os rótulos do eixo x
plt.xticks(rotation=90)

# Exibindo o gráfico
plt.show()

In [ ]:
# Plotagem da quantidade de tipos de vacinas utilizadas por pais

df = px.data.tips() # 'tips' para receber somente os tipos
fig = px.histogram(df_pandas, x="numero_de_tipos_de_vacina_usados", y= "pais") # Nome do dataframe e do eixo
fig.show() # Exibição

In [ ]:
# Plotagem dos países que iniciaram o período de vacinação primeiro

# Ordenando o DataFrame pela coluna "data_inicio_vacinacao"
df_sorted = df_pandas.sort_values(by='data_inicio_vacinacao')

# Plotando o gráfico de barras ordenado usando seaborn
sns.countplot(x='pais', data=df_sorted)

# Rotacionando os rótulos do eixo x
plt.xticks(rotation=90)

# Exibindo o gráfico
plt.show()


In [ ]:
# Plotagem do número total de pessoas com somente uma dose por país, ordenado pela ultima data de atualização

grafico = px.treemap(df_pandas, 
                     path=['pais', 'cumulativo_de_pessoas_com_uma_dose'], 
                     color='data_ultima_atualizacao',
                     color_continuous_scale='blues')

grafico.show()

# Preparação dos Arquivos

In [ ]:
# Salvando dataset tratado localmente, em formato 'csv'

df_pandas.to_csv('vaccination_data_treated.csv', index=False)

# Envio para o MongoDB

In [ ]:
# Criando conexão com o MongoDB

uri = "Mongo uri"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='Mongo Key')

In [ ]:
# Criando coleção para enviar para o MongoDB

db = client['covid_vaccination_data']
colecaotreated = db['covid_vaccination_data_treated']

In [ ]:
# Ao enviar o dataset para o MongoDB ocorreu um erro pois as colunas de data possuem valores nulos do tipo "NaT" (Not a time)
# Faz-se necessário entao substituir os valores "NaT" por valores padrões, nesse caso será "0000-00-00"
df_pandas = df_pandas.fillna({'data_ultima_atualizacao': 0000-00-00})
df_pandas = df_pandas.fillna({'data_inicio_vacinacao': 0000-00-00})

In [ ]:
# Enviando o Dataset tratado para o MongoDB

df_pandas.reset_index(drop=True)
df01 = df_pandas.to_dict("records")
colecaotreated.insert_many(df01)

In [ ]:
# Verificando se o arquivo foi enviado com sucesso
colecaotreated.count_documents({})
# Como podemos conferir o dataset foi enviado corretamente para o MongoDB

# FIM DO PROCESSO